### 기본 스태킹 모델

In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data=load_breast_cancer()

C:\Users\gommg\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\gommg\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
X_data=data.data
y_data=data.target

X_train,X_test,y_train,y_test=train_test_split(X_data,y_data,test_size=0.2,random_state=1)

In [5]:
#개별 분류 모델 설정
knn_clf=KNeighborsClassifier(n_neighbors=4)
rf_clf=RandomForestClassifier(n_estimators=100,random_state=1)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

#스태킹으로 만든 데이터를 학습,최종 예측할 모델
lr_final=LogisticRegression(C=10)

In [6]:
#개별 모델 학습
knn_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [9]:
knn_pred=knn_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
dt_pred=dt_clf.predict(X_test)
ada_pred=ada_clf.predict(X_test)

print(f'knn: {accuracy_score(y_test,knn_pred)}')
print(f'rf: {accuracy_score(y_test,rf_pred)}')
print(f'dt: {accuracy_score(y_test,dt_pred)}')
print(f'ada: {accuracy_score(y_test,ada_pred)}')

knn: 0.9035087719298246
rf: 0.956140350877193
dt: 0.956140350877193
ada: 0.9473684210526315


#### 개별 알고리즘의 예측값을 칼럼 레벨로 옆으로 붙여 피쳐 값으로 만들어 최종 모            델인 로지스틱 회귀에서 학습 데이터로 다시 사용한다.

In [11]:
pred=np.array([knn_pred,rf_pred,dt_pred,ada_pred])
print(pred.shape)

#tranpose해서 전치
pred=np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [12]:
lr_final.fit(pred,y_test)
final=lr_final.predict(pred)

print(f'final: {accuracy_score(y_test,final)}')

final: 0.9736842105263158


### CV기반 스태킹